## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re

## Part 1: Upload the data

In [12]:
# Visualize the data
data = pd.read_excel("/Users/Rachad/Desktop/Rachad/Ironhack/Modules_Labs_Ironhack/Module1/lab33/example_data_cleaning.xlsx")
data.head()

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
0,4,34985,1923.000000,5670,manager,78,Low
1,16,34997,1923.000000,2399090,developer,78,High
2,25,35006,1923.000000,33050,HR,78,High
3,12,34993,1939.142857,23430,professor,78,Low
4,21,35002,1939.142857,16770,manager,78,Low


## Part 2: Analyze your data and create a plan for data preparation

First we notice that the birth year is a float and need to be transformed into integers

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TransactionID  30 non-null     int64  
 1   ClientID       30 non-null     int64  
 2   BirthYear      28 non-null     float64
 3   Amount         30 non-null     int64  
 4   Profession     28 non-null     object 
 5   Department     30 non-null     int64  
 6   Risk           30 non-null     object 
dtypes: float64(1), int64(4), object(2)
memory usage: 1.8+ KB


With descriptive stats We can already notice that the std is too high especially compared to the mean. 
This will result in a very high variance. This shows that there is outliers that should be evaluated. 

In [5]:
# Start by calculating the descriptive statistics 
DescStats = data["Amount"].describe()
DescStats
 

count    3.000000e+01
mean     1.010097e+05
std      4.342616e+05
min      1.230000e+03
25%      1.233000e+04
50%      1.899000e+04
75%      2.953500e+04
max      2.399090e+06
Name: Amount, dtype: float64

##### Planification for the cleaning:
#1- Examining Data for Potential Issues and remove special characters and clean categorical variables..
#2- Identify and fill in missing values.
#3- Correct incorrect data types.
#4- Remove low variance columns.
#5- Identify potential outliers.
#6- Identify and remove duplicate records.

## Part 3: Data cleansing (missing values, outliers, duplicates, data consistently)

#### 1- Examining Data for Potential Issues and remove special characters and clean categorical variables.

In [13]:
#Start by removing trailing spaces at the end of the column names if there are any
data.columns = data.columns.str.rstrip(' ')

In [14]:
#Check the unique values and see if duplicates are due to different spelling or characters 
print('In the column Profession : ', data['Profession'].unique())
print('\n')
print('In the column Profession : ', data['Risk'].unique())

In the column Profession :  ['manager' 'developer' 'HR' 'professor' 'researcher' 'student' 'barmen'
 'Manager' 'bdm' nan 'hr' 'etudient' 'BDM' 'Hairdresser' 'Student'
 'Driver' 'sailer']


In the column Profession :  ['Low' 'High' 'Medium']


In [15]:
#The idea is to make all duplicates written the same way in order to be recognized as same with all functions
data['Profession'] = data['Profession'].str.replace('etudient', 'Student')
data['Profession'] = data['Profession'].str.capitalize()
data['Profession'] = data['Profession'].str.replace('Hr', 'HR')
print('repaired column Profession : ', data['Profession'].unique())

repaired column Profession :  ['Manager' 'Developer' 'HR' 'Professor' 'Researcher' 'Student' 'Barmen'
 'Bdm' nan 'Hairdresser' 'Driver' 'Sailer']


#### 2- Identify and fill in missing values.

In [11]:
# missing values: we can notice there is two missing values for BirthYear and two for Profession
data.isnull().sum()

TransactionID    0
ClientID         0
BirthYear        0
Amount           0
Profession       2
Department       0
Risk             0
dtype: int64

##### Evaluating the missing value in Profession and BirthYear


In [16]:
# Check using the clientID if the missing values belong to clients that already have there information in the table
null_displ = data[(data['Profession'].isnull()==True) | (data['BirthYear'].isnull()==True)]
null_displ

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
15,28,35008,1967.0,46370,NaN,78,High
16,29,35008,1976.0,50810,NaN,78,Medium
28,22,34987,NaN,18990,Sailer,78,High
29,7,34988,NaN,12330,Manager,78,Medium


We can notice that the clientIDs 35008, 34987 and 34988 is already mentionned in the table.

    For the client 35008 we can notice that a typo of the year of birth was generated : 1976 instead of 1967. 
    Based on The data the profession of this client seems to be Bdm

     For the client 34987 we can notice that the profession seems to be different Manager and Sailer. 
     I have a doubt about imputng the same year as the other two rows belonging to the same clientID

     For the client 34988 we can notice two rows with same ID and same Profession. Seems to be the same person.  
     I think we can replace the null value by the year 1999. 

In [17]:
test = data[(data['ClientID']==35008) | (data['ClientID']==34987) | (data['ClientID']==34988)]
test.sort_values("ClientID")

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
11,6,34987,1967.0,10110,Manager,78,Medium
12,15,34987,1967.0,30090,Manager,78,Low
28,22,34987,NaN,18990,Sailer,78,High
22,23,34988,1999.0,25650,Manager,78,Low
29,7,34988,NaN,12330,Manager,78,Medium
14,27,35008,1967.0,41930,Bdm,78,Low
15,28,35008,1967.0,46370,NaN,78,High
16,29,35008,1976.0,50810,NaN,78,Medium


We can fill the missing values in the profession column

In [20]:
data[['Profession']] = data[['Profession']].fillna('Bdm')


We can fill the missing values in the profession column

In [19]:
null_BirthYear = data[(data['BirthYear'].isnull()==True)]
print(null_BirthYear)
data2=data.query('ClientID == 34988 | ClientID == 34987')
data2
data.at[28,'BirthYear']=1967
data.at[29,'BirthYear']=1999
data

    TransactionID  ClientID  BirthYear  Amount Profession  Department    Risk
28             22     34987        NaN   18990     Sailer          78    High
29              7     34988        NaN   12330    Manager          78  Medium


,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk
0,4,34985,1923.000000,5670,Manager,78,Low
1,16,34997,1923.000000,2399090,Developer,78,High
2,25,35006,1923.000000,33050,HR,78,High
3,12,34993,1939.142857,23430,Professor,78,Low
4,21,35002,1939.142857,16770,Manager,78,Low
5,11,34992,1943.857143,21210,Researcher,78,Medium
6,20,35001,1943.857143,14550,Student,78,Medium
7,3,34984,1945.000000,3450,Student,78,Medium
8,19,35000,1948.571429,12330,Barmen,78,High
9,9,34990,1953.285714,16770,Manager,78,Medium


#### 3- Correct incorrect data types.

We can notice that BirthYear column is flaot so we need to transform it into integers

In [9]:
data['BirthYear'].dtype


dtype('int32')

##### 4- Remove low variance columns.

In [21]:
low_variance = []

for col in data._get_numeric_data(): #because stats only work with numbers
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)

print(low_variance)

['Department']


So we can see that Department is a low variance column so we can remove it if we want. I choose to keep it in the dataset

#### 5- Identify potential outliers.

In [22]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
TransactionID,30.0,15.500000,8.803408,1.0,8.250000,15.5,22.75,30.0
ClientID,30.0,34993.833333,8.132876,34982.0,34987.250000,34991.5,34999.75,35008.0
BirthYear,30.0,1969.295238,27.560513,1923.0,1945.892857,1967.0,1988.00,2017.0
Amount,30.0,101009.666667,434261.568869,1230.0,12330.000000,18990.0,29535.00,2399090.0
Department,30.0,78.000000,0.000000,78.0,78.000000,78.0,78.00,78.0


In [23]:
stats = data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
TransactionID,30.0,15.500000,8.803408,1.0,8.250000,15.5,22.75,30.0,14.500000
ClientID,30.0,34993.833333,8.132876,34982.0,34987.250000,34991.5,34999.75,35008.0,12.500000
BirthYear,30.0,1969.295238,27.560513,1923.0,1945.892857,1967.0,1988.00,2017.0,42.107143
Amount,30.0,101009.666667,434261.568869,1230.0,12330.000000,18990.0,29535.00,2399090.0,17205.000000
Department,30.0,78.000000,0.000000,78.0,78.000000,78.0,78.00,78.0,0.000000


In [24]:
outliers = pd.DataFrame(columns=data.columns)

for col in stats.index:  
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) | 
                   (data[col] > upper)].copy() 
    results['Outlier'] = col
    outliers = outliers.append(results)

C:\Users\Rachad\AppData\Local\Temp\ipykernel_2760\182753042.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers = outliers.append(results)
C:\Users\Rachad\AppData\Local\Temp\ipykernel_2760\182753042.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers = outliers.append(results)
C:\Users\Rachad\AppData\Local\Temp\ipykernel_2760\182753042.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers = outliers.append(results)
C:\Users\Rachad\AppData\Local\Temp\ipykernel_2760\182753042.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers = outliers.append(results)
C:\Users\Rachad\AppData\Local\Temp\ipykernel_276

In [25]:
outliers

,TransactionID,ClientID,BirthYear,Amount,Profession,Department,Risk,Outlier
1,16,34997,1923.0,2399090,Developer,78,High,Amount


#### 6- Identify and remove duplicate records.

No rows duplicates were observed

In [27]:
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  0


## Part 4: Encode categorical data

Encoding the categorical data Profession and Risk

In [28]:
pd.get_dummies(data)

,TransactionID,ClientID,BirthYear,Amount,Department,Profession_Barmen,Profession_Bdm,Profession_Developer,Profession_Driver,Profession_HR,Profession_Hairdresser,Profession_Manager,Profession_Professor,Profession_Researcher,Profession_Sailer,Profession_Student,Risk_High,Risk_Low,Risk_Medium
0,4,34985,1923.000000,5670,78,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1,16,34997,1923.000000,2399090,78,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,25,35006,1923.000000,33050,78,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,12,34993,1939.142857,23430,78,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,21,35002,1939.142857,16770,78,0,0,0,0,0,0,1,0,0,0,0,0,1,0
5,11,34992,1943.857143,21210,78,0,0,0,0,0,0,0,0,1,0,0,0,0,1
6,20,35001,1943.857143,14550,78,0,0,0,0,0,0,0,0,0,0,1,0,0,1
7,3,34984,1945.000000,3450,78,0,0,0,0,0,0,0,0,0,0,1,0,0,1
8,19,35000,1948.571429,12330,78,1,0,0,0,0,0,0,0,0,0,0,1,0,0
9,9,34990,1953.285714,16770,78,0,0,0,0,0,0,1,0,0,0,0,0,0,1
